<a href="https://colab.research.google.com/github/gmu1974/Curso-Data-Engineering/blob/master/PrimeraEntrega_GastonUrbina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##*** Paso 1: Extraccion data API***


In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import psycopg2


In [ ]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Installing the Python connector from PyPI
!pip install redshift_connector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.4 MB/s eta 0:00:00


##*** Efectuamos la conexiones a la API ***

Se trata de la cantidad de estupefacientes incautados por Colombia en los ulimos 23 años, por cada municipio y ciudad


In [ ]:
import requests
import json
import pandas as pd
import numpy as np


In [ ]:
response = requests.get(url='https://www.datos.gov.co/resource/kk69-w2jj.json')
response

<Response [200]>

In [ ]:
response.text

'[{"departamento":"AMAZONAS","municipio":"LETICIA (CT)","codigo_dane":"91001000","clase_bien":"COCAINA","fecha_hecho":"1/01/2010","cantidad":"8.825"}\n,{"departamento":"ANTIOQUIA","municipio":"AMALFI","codigo_dane":"5031000","clase_bien":"MARIHUANA","fecha_hecho":"1/01/2010","cantidad":"16"}\n,{"departamento":"ANTIOQUIA","municipio":"ANDES","codigo_dane":"5034000","clase_bien":"MARIHUANA","fecha_hecho":"1/01/2010","cantidad":"10"}\n,{"departamento":"ANTIOQUIA","municipio":"ANGELÓPOLIS","codigo_dane":"5036000","clase_bien":"MARIHUANA","fecha_hecho":"1/01/2010","cantidad":"80"}\n,{"departamento":"ANTIOQUIA","municipio":"ARBOLETES","codigo_dane":"5051000","clase_bien":"MARIHUANA","fecha_hecho":"1/01/2010","cantidad":"500"}\n,{"departamento":"ANTIOQUIA","municipio":"BELLO","codigo_dane":"5088000","clase_bien":"MARIHUANA","fecha_hecho":"1/01/2010","cantidad":"10"}\n,{"departamento":"ANTIOQUIA","municipio":"BETANIA","codigo_dane":"5091000","clase_bien":"COCAINA","fecha_hecho":"1/01/2010","ca

In [ ]:
data = json.loads(response.text)
data

[{'departamento': 'AMAZONAS',
  'municipio': 'LETICIA (CT)',
  'codigo_dane': '91001000',
  'clase_bien': 'COCAINA',
  'fecha_hecho': '1/01/2010',
  'cantidad': '8.825'},
 {'departamento': 'ANTIOQUIA',
  'municipio': 'AMALFI',
  'codigo_dane': '5031000',
  'clase_bien': 'MARIHUANA',
  'fecha_hecho': '1/01/2010',
  'cantidad': '16'},
 {'departamento': 'ANTIOQUIA',
  'municipio': 'ANDES',
  'codigo_dane': '5034000',
  'clase_bien': 'MARIHUANA',
  'fecha_hecho': '1/01/2010',
  'cantidad': '10'},
 {'departamento': 'ANTIOQUIA',
  'municipio': 'ANGELÓPOLIS',
  'codigo_dane': '5036000',
  'clase_bien': 'MARIHUANA',
  'fecha_hecho': '1/01/2010',
  'cantidad': '80'},
 {'departamento': 'ANTIOQUIA',
  'municipio': 'ARBOLETES',
  'codigo_dane': '5051000',
  'clase_bien': 'MARIHUANA',
  'fecha_hecho': '1/01/2010',
  'cantidad': '500'},
 {'departamento': 'ANTIOQUIA',
  'municipio': 'BELLO',
  'codigo_dane': '5088000',
  'clase_bien': 'MARIHUANA',
  'fecha_hecho': '1/01/2010',
  'cantidad': '10'},
 {

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   departamento  1000 non-null   object
 1   municipio     1000 non-null   object
 2   codigo_dane   1000 non-null   object
 3   clase_bien    1000 non-null   object
 4   fecha_hecho   1000 non-null   object
 5   cantidad      1000 non-null   object
dtypes: object(6)
memory usage: 47.0+ KB


##*** ETL de la data ***

In [ ]:
df['indice'] = df.index

In [ ]:
df['cantidad'] = df['cantidad'].astype('float64')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   departamento  1000 non-null   object 
 1   municipio     1000 non-null   object 
 2   codigo_dane   1000 non-null   object 
 3   clase_bien    1000 non-null   object 
 4   fecha_hecho   1000 non-null   object 
 5   cantidad      1000 non-null   float64
 6   indice        1000 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 54.8+ KB


##*** Visualizamos el Dataframe ***

In [ ]:
df.head()

,departamento,municipio,codigo_dane,clase_bien,fecha_hecho,cantidad,indice
0,AMAZONAS,LETICIA (CT),91001000,COCAINA,1/01/2010,8.825,0
1,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16.000,1
2,ANTIOQUIA,ANDES,5034000,MARIHUANA,1/01/2010,10.000,2
3,ANTIOQUIA,ANGELÓPOLIS,5036000,MARIHUANA,1/01/2010,80.000,3
4,ANTIOQUIA,ARBOLETES,5051000,MARIHUANA,1/01/2010,500.000,4


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   departamento  1000 non-null   object 
 1   municipio     1000 non-null   object 
 2   codigo_dane   1000 non-null   object 
 3   clase_bien    1000 non-null   object 
 4   fecha_hecho   1000 non-null   object 
 5   cantidad      1000 non-null   float64
 6   indice        1000 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 54.8+ KB


##*** Verificamos si existen datos nulos ***

In [ ]:
df_check=df.isnull().sum()

print(df_check)

departamento    0
municipio       0
codigo_dane     0
clase_bien      0
fecha_hecho     0
cantidad        0
indice          0
dtype: int64


##*** Verificamos si existen datos duplicados ***
Como puede observarse , no los tenemos...

In [ ]:
new_val = df[df.duplicated(keep = 'last')]

print("Duplicate Rows :")
print(new_val)

Duplicate Rows :
Empty DataFrame
Columns: [departamento, municipio, codigo_dane, clase_bien, fecha_hecho, cantidad, indice]
Index: []


##*** Inventamos unos registros para practicar el ETL ***,
verificamos que se hayan agregado correctamente.... (sumamos 2 registros iguales)

In [ ]:
df=df.append({'departamento':'ANTIOQUIA','municipio':	'AMALFI', 'codigo_dane': '5031000', 'clase_bien':	'MARIHUANA', 'fecha_hecho': '1/01/2010', 'cantidad': 16.000, 'indice': '1000'}, ignore_index=True)

<ipython-input-40-319acc574fdf>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append({'departamento':'ANTIOQUIA','municipio':	'AMALFI', 'codigo_dane': '5031000', 'clase_bien':	'MARIHUANA', 'fecha_hecho': '1/01/2010', 'cantidad': 16.000, 'indice': '1000'}, ignore_index=True)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1003 entries, 0 to 1002
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   departamento  1003 non-null   object
 1   municipio     1003 non-null   object
 2   codigo_dane   1003 non-null   object
 3   clase_bien    1003 non-null   object
 4   fecha_hecho   1003 non-null   object
 5   cantidad      1003 non-null   object
 6   indice        1003 non-null   object
dtypes: object(7)
memory usage: 55.0+ KB


***Acabamos de agregar registros, uno de ellos ha sido duplicado para practicar las funcionalidades de Pandas ***. Volvemos a corroborar su existencia

In [ ]:
df.tail()

,departamento,municipio,codigo_dane,clase_bien,fecha_hecho,cantidad,indice
998,NARIÑO,NARIÑO,52480000,MARIHUANA,8/01/2010,40.0,998
999,NARIÑO,PASTO (CT),52001000,BASUCO,8/01/2010,60.0,999
1000,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16.000,1000
1001,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16.0,1000
1002,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16.0,1000


In [ ]:
new_val = df[df.duplicated(keep = 'last')]

print("Duplicate Rows :")
print(new_val)

Duplicate Rows :
     departamento municipio codigo_dane clase_bien fecha_hecho cantidad indice
1001    ANTIOQUIA    AMALFI     5031000  MARIHUANA   1/01/2010     16.0   1000


In [ ]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
998     False
999     False
1000    False
1001    False
1002     True
Length: 1003, dtype: bool

##*** Finalmente eliminamos registros duplicados***,

In [ ]:
df = df.drop_duplicates()
df

,departamento,municipio,codigo_dane,clase_bien,fecha_hecho,cantidad,indice
0,AMAZONAS,LETICIA (CT),91001000,COCAINA,1/01/2010,8.825,0
1,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16.0,1
2,ANTIOQUIA,ANDES,5034000,MARIHUANA,1/01/2010,10.0,2
3,ANTIOQUIA,ANGELÓPOLIS,5036000,MARIHUANA,1/01/2010,80.0,3
4,ANTIOQUIA,ARBOLETES,5051000,MARIHUANA,1/01/2010,500.0,4
...,...,...,...,...,...,...,...
997,NARIÑO,LEIVA,52405000,MARIHUANA,8/01/2010,80.0,997
998,NARIÑO,NARIÑO,52480000,MARIHUANA,8/01/2010,40.0,998
999,NARIÑO,PASTO (CT),52001000,BASUCO,8/01/2010,60.0,999
1000,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16.000,1000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1002 entries, 0 to 1001
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   departamento  1002 non-null   object
 1   municipio     1002 non-null   object
 2   codigo_dane   1002 non-null   object
 3   clase_bien    1002 non-null   object
 4   fecha_hecho   1002 non-null   object
 5   cantidad      1002 non-null   object
 6   indice        1002 non-null   object
dtypes: object(7)
memory usage: 62.6+ KB


##*** Establecemos las conexiones en Redshift ***

In [ ]:
#connect to the cluster, redshift
import redshift_connector
conn = redshift_connector.connect(
     host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
     database='data-engineer-database',
     port=5439,
     user='gmu1974_coderhouse',
     password='v4vE3FEfl36t'
  )

##*** Creamos las tablas de destino en Redshift ***

In [ ]:
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname='data-engineer-database',
        user='gmu1974_coderhouse',
        password='v4vE3FEfl36t',
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [ ]:

#CREATE TABLE estupefacientes (
  #departamento VARCHAR(50)	
  #municipio	VARCHAR(50)
  #codigo_dane	VARCHAR(50)
  #clase_bien	VARCHAR(50)
  #fecha_hecho	DATE
  #cantidad	INT
  #indice INT PRIMAREY KEY
#);

In [ ]:
conn

<connection object at 0x7fb9d0be7d80; dsn: 'user=gmu1974_coderhouse password=xxx dbname=data-engineer-database host=data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com port=5439', closed: 0>

##*** Verificamos que las tablas existan ***

In [ ]:
cur = conn.cursor()
# Execute a SQL query to select data from a table
cur.execute("SELECT * FROM estupefacientes")
# Fetch the results
results = cur.fetchall()
results

[('AMAZONAS',
  'LETICIA (CT)',
  '91001000',
  'COCAINA',
  datetime.date(2010, 1, 1),
  9,
  0),
 ('ANTIOQUIA',
  'AMALFI',
  '5031000',
  'MARIHUANA',
  datetime.date(2010, 1, 1),
  16,
  1),
 ('ANTIOQUIA',
  'ANDES',
  '5034000',
  'MARIHUANA',
  datetime.date(2010, 1, 1),
  10,
  2),
 ('ANTIOQUIA',
  'ANGELÓPOLIS',
  '5036000',
  'MARIHUANA',
  datetime.date(2010, 1, 1),
  80,
  3),
 ('ANTIOQUIA',
  'ARBOLETES',
  '5051000',
  'MARIHUANA',
  datetime.date(2010, 1, 1),
  500,
  4),
 ('ANTIOQUIA',
  'BELLO',
  '5088000',
  'MARIHUANA',
  datetime.date(2010, 1, 1),
  10,
  5),
 ('ANTIOQUIA',
  'BETANIA',
  '5091000',
  'COCAINA',
  datetime.date(2010, 1, 1),
  4,
  6),
 ('ANTIOQUIA',
  'BETANIA',
  '5091000',
  'MARIHUANA',
  datetime.date(2010, 1, 1),
  60,
  7),
 ('ANTIOQUIA',
  'CÁCERES',
  '5120000',
  'COCAINA',
  datetime.date(2010, 1, 1),
  2,
  8),
 ('ANTIOQUIA',
  'CÁCERES',
  '5120000',
  'MARIHUANA',
  datetime.date(2010, 1, 1),
  67,
  9),
 ('ANTIOQUIA',
  'CISNEROS',
  '

##*** Finalmente enviamos los datos de la Tabla a su equivalente Redshift ***

In [ ]:
df.head()

,departamento,municipio,codigo_dane,clase_bien,fecha_hecho,cantidad,indice
0,AMAZONAS,LETICIA (CT),91001000,COCAINA,1/01/2010,8.825,0
1,ANTIOQUIA,AMALFI,5031000,MARIHUANA,1/01/2010,16.000,1
2,ANTIOQUIA,ANDES,5034000,MARIHUANA,1/01/2010,10.000,2
3,ANTIOQUIA,ANGELÓPOLIS,5036000,MARIHUANA,1/01/2010,80.000,3
4,ANTIOQUIA,ARBOLETES,5051000,MARIHUANA,1/01/2010,500.000,4


In [ ]:
from psycopg2.extras import execute_values
cur = conn.cursor()
# Define the table name
table_name = 'estupefacientes'
# Define the columns you want to insert data into
columns = ['departamento','municipio','codigo_dane','clase_bien','fecha_hecho','cantidad','indice']
# Generate 
values = [tuple(x) for x in df.to_numpy()]
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"
# Execute the INSERT statement using execute_values
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")
# Close the cursor and connection
#cur.close()
#conn.close()

In [ ]:
conn.close()
cur.close()